# Sequence Analysis with Python


In [196]:
from collections import defaultdict
from itertools import product
import pandas as pd
import numpy as np
from numpy.random import choice
import re
from collections import defaultdict, Counter

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [197]:
def dna_to_rna(s):

    dic =  {'T': 'U'}
    return "".join(dic["T"] if c in dic else c for c in s)
    
if __name__ == '__main__':
    print(dna_to_rna("AACGTGATTTC"))

AACGUGAUUUC


## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [198]:
def get_dict():
    #opening the codon usage table (must be saved to your src - pls see .README)
    with open("codonusagetable.html", "r") as html_file:
        content = html_file.read()
    dic = {}
    pattern = r"([ACGU]{3,3}) ([A-Z\*])"
    matches = re.findall(pattern, content)
    for (codon, amino_acid) in matches:
        dic[codon] = amino_acid
        
    return dic
    
if __name__ == '__main__':
    codon_to_aa = get_dict()
    print(codon_to_aa)

{'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}


3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [199]:
def get_dict_list():
    dic2 = {}
    dic = get_dict()
    for item in dic.items(): 
        if item[1] in dic2.keys(): 
            dic2[item[1]].append(item[0])
        else: 
            dic2[item[1]] = [item[0]]
            
    return dic2
    
if __name__ == '__main__':
    aa_to_codons = get_dict_list()
    print(aa_to_codons)

{'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UGA', 'UAG'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}


With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [200]:
def rna_to_prot(s):
    dic = get_dict()
    
    #For each codon (size 3), we find the corresponding amino acid from the dictionary. 
    #Then we join the aminoacids to form a string (protein).
    return "".join([dic[s[i:i+3]] for i in range(0, len(s), 3)])

def dna_to_prot(s):
    return rna_to_prot(dna_to_rna(s))

if __name__ == '__main__':
    print(dna_to_prot("ATGATATCATCGACGATGTAG")) 

MISSTM*


You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function.

In [201]:
def get_probabability_dict():
    #opens the codon usage table (must be saved to your src!)
    #and then finds the RNA sequence and its frequences. 
    with open("codonusagetable.html", "r") as html_file:
        context = html_file.read()
    
    pattern = r"([ACGU]{3,3})\s+[A-Z\*]\s+\d+\.\d+\s+\d+\.\d+\s+\(\s*(\d+)\)"
    matches = re.findall(pattern, context)
    count = {}
    for (codon, n) in matches:
        count[codon] = int(n)
        
    amino_acid_encoders = get_dict_list()
    prob = {}    
    for encoders in amino_acid_encoders.values():
        counts = [count[codon] for codon in encoders]
        total = sum(counts)
        for (codon, n) in zip(encoders, counts):
            prob[codon] = n/total
    return prob
    
if __name__ == '__main__':
    codon_to_prob = get_probabability_dict()
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i*6:6+i*6]
        ))

AAA: 0.434049	AAC: 0.529633	AAG: 0.565951	AAU: 0.470367	ACA: 0.284188	ACC: 0.355232
ACG: 0.113812	ACU: 0.246769	AGA: 0.214658	AGC: 0.239938	AGG: 0.211091	AGU: 0.149602
AUA: 0.169062	AUC: 0.469866	AUG: 1.000000	AUU: 0.361072	CAA: 0.265017	CAC: 0.581485
CAG: 0.734983	CAU: 0.418515	CCA: 0.276603	CCC: 0.323470	CCG: 0.113196	CCU: 0.286731
CGA: 0.108812	CGC: 0.183777	CGG: 0.201554	CGU: 0.080108	CUA: 0.071380	CUC: 0.195577
CUG: 0.395702	CUU: 0.131716	GAA: 0.422453	GAC: 0.535458	GAG: 0.577547	GAU: 0.464542
GCA: 0.228121	GCC: 0.399781	GCG: 0.106176	GCU: 0.265922	GGA: 0.249922	GGC: 0.337109
GGG: 0.249882	GGU: 0.163087	GUA: 0.116577	GUC: 0.238306	GUG: 0.463346	GUU: 0.181770
UAA: 0.297019	UAC: 0.556662	UAG: 0.236738	UAU: 0.443338	UCA: 0.150517	UCC: 0.217960
UCG: 0.054398	UCU: 0.187586	UGA: 0.466243	UGC: 0.543843	UGG: 1.000000	UGU: 0.456157
UUA: 0.076568	UUC: 0.535866	UUG: 0.129058	UUU: 0.464134


Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [202]:
class ProteinToMaxRNA:
    
    def __init__(self):
        pass

    #converts a protein sequence into the most likely RNA sequence to be the source of this protein
    def convert(self, s):
        aminos = list(s)
        dl = get_dict_list()
        
        #dic7 includes only needed RNA amino acids and their sequences
        dic7 = {}
        for amino in aminos: 
            dic7[amino] = dl[amino]
        
        #finding the most probable sequences for amino acids
        chosenseqs = []
        probdic = get_probabability_dict()
        for item in dic7.items(): 
            biggestval = 0.0
            chosenseq = ""
            for sequence in item[1]: 
                if probdic[sequence] > biggestval:
                    biggestval = probdic[sequence]
                    chosenseq = sequence

            chosenseqs.append(chosenseq)
                
        #forming a RNA sequence: 
        a = ""
        for seq in chosenseqs: 
            a = a+seq
        
        return a


if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print(protein_to_rna.convert("LTPIQNRA"))

CUGACCCCCAUCCAGAACAGAGCC


Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [203]:
def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    
    random = np.random.uniform() #random number [0-1]
    parts = len(dist)
    bounds = 100/parts 
    
    #adding keys to a list  
    keys = []
    for key in dist.keys(): 
        keys.append(key)
                
    #let's choose the suitable amino acid by taking advance of random 
    #that defines which amino acid to choose from the probability distribution. 
    i = 0
    k = keys[i] #the first letter
    arvo = dist[k] 
    while i <= parts: 
        if random == 1.0:
            valittu = keys[parts-1]
            break
        if random < arvo: 
            valittu = keys[i]
            break
        else: 
            i = i+1
            arvo = arvo + dist[keys[i]]
            
    return valittu


if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(", ".join(random_event(distribution) for _ in range(29)))

C, C, C, C, A, T, T, T, C, T, C, T, A, C, G, A, G, T, C, T, T, C, C, G, T, C, T, C, T


With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [204]:
class ProteinToRandomRNA(object):
    
    def __init__(self):
        pass

    #function returns random RNA-sequence based on given protein and probabilities
    def convert(self, s):
        letters = list(s)
        
        #finding each protein its sequence
        dictlist = get_dict_list()
        
        valuelists = []
        for letter in letters: 
            valuelists.append(dictlist[letter])
            
        #creating a distribution 
        distribution = dict(zip((letters), valuelists))
        
        #finding probabilities for sequences 
        probs = get_probabability_dict()
        
        i = 0
        proteinsequence = ""
        for letter in letters: 
            #viedään tarvittavien sekvenssien todennäköisyydet listalle seqprobs: 
            seqprobs = []
            lista = distribution[letters[i]]
            for seq in lista: 
                seqprobs.append(probs[seq])
            
            #creating a probability distribtuion for letters: 
            distribution2 = dict(zip(valuelists[i], seqprobs))
                
            #ceating random sequence based on probabilities
            chosen = random_event(distribution2)
            proteinsequence = proteinsequence + chosen 
            i = i+1
                    
        return proteinsequence
    
        
if __name__ == '__main__':
    protein_to_random_codons = ProteinToRandomRNA()
    print(protein_to_random_codons.convert("LTPIQNRA"))

CUGACUCCAAUCCAAAACCGGGCC


## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [205]:
def sliding_window(s, k):
    """
    This function returns a generator that can be iterated over all
    starting position of a k-window in the sequence.
    For each starting position the generator returns the nucleotide frequencies
    in the window as a dictionary.
    """
    i = 0
    while i<(len(s)-(k-1)):
        #splitting the string to 4 character strings
        patka = s[i: (i+k)]
        i = i+1
        
        #counting each letter its frequency in this 4 character string --> to dictionary 
        dic = {"A":0, "C":0, "G":0, "T":0}
        for l in patka: 
            if l == "A":
                dic["A"] = (dic["A"]+1)
            elif l == "C":
                dic["C"] = (dic["C"]+1)
            elif l == "G": 
                dic["G"] = (dic["G"]+1)
            elif l == "T": 
                dic["T"] = (dic["T"]+1)
        
        yield dic
        
    
if __name__ == '__main__':
    s = "TCCCGACGGCCTTGCC"
    for d in sliding_window(s, 4):
        print(d)

{'A': 0, 'C': 3, 'G': 0, 'T': 1}
{'A': 0, 'C': 3, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}
{'A': 0, 'C': 2, 'G': 0, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}


 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [206]:
#sequence s is iterated k letters at a time and
#following letters are returned as a dict
def context_list(s, k):
    dict = {}
    for i in range(k, len(s)):
        a = s[i-k:i]
        c = s[i]
        dict[a] = dict.get(a, "") + c
    return dict
    
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATCTAG"
    d = context_list(s, k)
    print(d)

{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}


11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [207]:
def context_probabilities(s, k):
    
    if k == 0: 
        #counting probability of the letter in s
        setti = set(s) 
        listana = list(s)
        dic1 = {}
        for let in setti: 
            esiintyy = 0
            for i,m in enumerate(re.finditer(let, s)): 
                esiintyy = esiintyy + 1
                proba = esiintyy/len(s)
                dic1[let] = proba

        return dic1

    else:
        contlist = context_list(s, k)
        dic = {}

        for item in contlist.items():  
            for letter in sorted(item[1]): 
                total = len(item[1])
                list_set = set(item[1]) 
                esiintymat = 0
                for i,m in enumerate(re.finditer(letter, item[1])): 
                    esiintymat = esiintymat + 1
                    prob = esiintymat/total
                    if item[0] in dic.keys():
                        dic[item[0]][letter] = prob
                    else: 
                        dic[item[0]] = {letter: prob}            

        return dic
    
if __name__ == '__main__':
    s = "ATGATATCATCGACGATGTAG"
    k = 0
    d = context_probabilities(s, k)
    print(d)
    k = 2
    d = context_probabilities(s, k)
    print(d)

{'C': 0.14285714285714285, 'T': 0.2857142857142857, 'A': 0.3333333333333333, 'G': 0.23809523809523808}
{'AT': {'A': 0.2, 'C': 0.4, 'G': 0.4}, 'TG': {'A': 0.5, 'T': 0.5}, 'GA': {'C': 0.3333333333333333, 'T': 0.6666666666666666}, 'TA': {'G': 0.5, 'T': 0.5}, 'TC': {'A': 0.5, 'G': 0.5}, 'CA': {'T': 1.0}, 'CG': {'A': 1.0}, 'AC': {'G': 1.0}, 'GT': {'A': 1.0}}


12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [208]:
class MarkovChain:
    
    def __init__(self, zeroth, kth, k=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
    
    def generate(self, n, seed=None):
        
        #first and second group of the sequence: 
        first = random_event(zeroth)
        seconds = []
        for key in kth.keys(): 
            if key.startswith(first):
                seconds.append(key)
        dic = {}
        for s in seconds: 
            dic[s] = 1/len(seconds)
            
        #starting codon 
        starter = random_event(dic)

        #rest of the DNA (n-k letters)
        dna = starter 
        i = 0
        while i < n-k:
            nextletter = random_event(kth[dna[-k:]])
            dna = dna + nextletter
            i = i+1
            
        return dna

if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
    n = 10    
    seed = 0
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n, seed))

CGATGATCGA


If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [209]:
import itertools

def context_pseudo_probabilities(s, k):
    letters = context_list(s, k)
    for slist in product("ACGT", repeat = k):
        s = "".join(slist)
        letters[s] = letters.get(s, "") + "ACGT"
    dist = {}
    for s, cs in letters.items():
        dist[s] = {c: count / len(cs) for c, count in Counter(cs).items()}
    return dist
    
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth = context_pseudo_probabilities(s, k)
    zeroth = context_pseudo_probabilities(s, 0)[""]
    print(f"zeroth: {zeroth}")
    print("\n".join(f"{k}: {dict(v)}" for k, v in kth.items()))
    
    print("\n", MarkovChain(zeroth, kth, k).generate(20))

zeroth: {'A': 0.32, 'T': 0.28, 'G': 0.24, 'C': 0.16}
AT: {'G': 0.3333333333333333, 'A': 0.2222222222222222, 'C': 0.3333333333333333, 'T': 0.1111111111111111}
TG: {'A': 0.3333333333333333, 'T': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.16666666666666666}
GA: {'T': 0.42857142857142855, 'C': 0.2857142857142857, 'A': 0.14285714285714285, 'G': 0.14285714285714285}
TA: {'T': 0.3333333333333333, 'G': 0.3333333333333333, 'A': 0.16666666666666666, 'C': 0.16666666666666666}
TC: {'A': 0.3333333333333333, 'G': 0.3333333333333333, 'C': 0.16666666666666666, 'T': 0.16666666666666666}
CA: {'T': 0.4, 'A': 0.2, 'C': 0.2, 'G': 0.2}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
AC: {'G': 0.4, 'A': 0.2, 'C': 0.2, 'T': 0.2}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [225]:
class MarkovProb:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth 
        self.kth = kth 
        
    #counts the probability of the DNA sequence s
    #the probability is counted multiplying probabilities so that
    #keys probability is l/amount of keys and values probability its value 
    def probability(self, s):
        probs = context_pseudo_probabilities(s, k) 
        tulo = 1
        for letter in s[0:k]:
            tulo = tulo * zeroth[letter]
                
        #the probability of the starting codon: 
        ppatka = tulo
        i = 0
        probabilities = []
        while i<(len(s)-(k)):
            patka = s[i: (i+k)]
            seuraava = s[i+k] 
            t = probs[patka][seuraava]
            probabilities.append(t)
            i = i+1
           
        product = 1
        for x in probabilities:
            product = product * x
        product = product * ppatka
        
        return product

    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovProb(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Probability of sequence {s} is {mc.probability(s)}")

Probability of sequence ATGATATCATCGACGATGTAG is 2.831270190340018e-10


With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [226]:
import math as math

class MarkovLog(object):
    
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
    
    def log_probability(self, s):
        probs = context_pseudo_probabilities(s, k)
        
        #multiplying the probabilities of the starting codon
        tulo = 1
        for letter in s[0:k]:
            tulo = tulo * zeroth[letter]
                
        #so we get the probability of that codon: 
        ppatka = tulo
        
        #counting all the probabilities for s:   
        i = 0
        probabilities = []
        while i<(len(s)-(k)):
            patka = s[i: (i+k)]
            seuraava = s[i+k] 
            t = probs[patka][seuraava]
            probabilities.append(t)
            i = i+1
           
        product = 0
        for x in probabilities:
            product = product + math.log2(x)
        product = product + math.log2(ppatka)        
        
        return product
    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovLog(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {s} is {mc.log_probability(s)}")

Log probability of sequence ATGATATCATCGACGATGTAG is -31.71783151553831


Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [227]:
def better_context_probabilities(s, k):
    #Let's reuse the function from ex 13 with modifiactions.
    #no the funstion returns only needed (in s) probabilities
    probs = context_probabilities(s, k)
    
    dist = {}
    for W_list in product("ACGT", repeat=k):
        W = "".join(W_list)
        dist[W] = dict(zip("ACGT", [1]*4))
    for i in range(k, len(s)):
        W = s[i-k:i]
        c = s[i]
        dist[W][c] += 1
    for W, cs in dist.items():
        dist[W] = {c: count / sum(cs.values()) for c, count in cs.items()}
    return dist

    #finding every combination for letters ACGT 
    #and if combination is in s, adding that 
    for a in product("ACGT", repeat = k):
        uusi = ''.join(a)
        if uusi in probs.keys():
            pass
        else: 
            probs[uusi] = context_probabilities(s, 0)
            
    #finally adding probabilities for rest of the keys 
    for letters in probs.values():
        if "A" not in letters.keys():
            letters["A"] = 0.0
        if "C" not in letters.keys():
            letters["C"] = 0.0
        if "G" not in letters.keys():
            letters["G"] = 0.0
        if "T" not in letters.keys():
            letters["T"] = 0.0
        
    return probs

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = better_context_probabilities(s, k)
    print("\n".join(f"{k}: {v}" for k, v in d.items()))

AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666}
TG: {'A': 0.3333333333333333, 'C': 0.16666666666666